In [137]:
import pandas as pd
import numpy as np

In [138]:
df=pd.read_csv('../Data/Spotify_with_genre_clustered.csv')

In [139]:
df.head()

,artist_name,track_name,artist_pop,popularity,genre,track_uri,artist_uri,year,cluster
0,34,Preachin' To The Choir,0.37,0.000000,1,6JKj9seJCshLVCfBLFp7dS,55RI2GNCfyXr0f14uIdhwd,2017,6
1,2911,You Work Days I Work Nights,0.29,0.108696,2,7L7u4PrNt5WMtVI9lHWlLm,6k3UpifDbb2ox25edM5j3P,2012,0
2,485,Sera,0.44,0.336957,1,12EyOqMPMyAKlKSynICAGN,2qhLqZ1pkiUl5HNrU2Nz0R,2007,3
3,1268,Big Shit Poppin,0.69,0.206522,1,3tCDjKseLQJhShfbeg4m7K,4yBK75WVCQXej1p04GWqxH,2007,1
4,1525,No La Voy A Engañar,0.74,0.586957,1,0UICwSsSpUt98XkeCfupFn,5bSfBBCxY8QAk4Pifveisz,2012,1


In [140]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.client import SpotifyException

In [141]:
client_id = '6444952d25014134affbddb6854f27c7'
client_secret = '49c9c42e4b54426785ae5856b45b719e'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [142]:
# Function to fetch track info
def get_track_info(track_name):
    results = sp.search(q=track_name, limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_uri = track['uri'].split(':')[-1]  # Extracting the track URI
        artist_name = track['artists'][0]['name']  # Extracting the artist name
        release_date = track['album']['release_date'][:4]  # Extracting the year of release
        popularity = track['popularity']  # Track popularity
        artist_pop = sp.artist(track['artists'][0]['id'])['popularity']  # Artist popularity
        return track_uri, artist_name, release_date, popularity, artist_pop
    else:
        return None, None, None, None, None

In [143]:
# Function to fetch specific audio features
def get_audio_features(track_url):
    audio_features = sp.audio_features([track_url])
    if audio_features:
        # Extracting only desired audio features
        selected_features = {
            'danceability': audio_features[0]['danceability'],
            'energy': audio_features[0]['energy'],
            'key': audio_features[0]['key'],
            'loudness': audio_features[0]['loudness'],
            'mode': audio_features[0]['mode'],
            'speechiness': audio_features[0]['speechiness'],
            'acousticness': audio_features[0]['acousticness'],
            'instrumentalness': audio_features[0]['instrumentalness'],
            'liveness': audio_features[0]['liveness'],
            'valence': audio_features[0]['valence'],
            'tempo': audio_features[0]['tempo']
        }
        return selected_features
    else:
        return None

In [144]:
# Example usage
track_name = input("Enter the track name: ")
track_uri, artist_name, release_date, popularity, artist_pop = get_track_info(track_name)
if track_uri:
    print("Track URL:", track_uri)
    print("Artist:", artist_name)
    print("Release Year:", release_date)
    print("Track Popularity:", popularity)
    print("Artist Popularity:", artist_pop)
    audio_features = get_audio_features(track_uri)
    if audio_features:
        print("Audio Features:", audio_features)
    else:
        print("Failed to fetch audio features.")
else:
    print("Track not found.")

Track URL: 7MXVkk9YMctZqd1Srtv4MB
Artist: The Weeknd
Release Year: 2016
Track Popularity: 95
Artist Popularity: 95
Audio Features: {'danceability': 0.679, 'energy': 0.587, 'key': 7, 'loudness': -7.015, 'mode': 1, 'speechiness': 0.276, 'acousticness': 0.141, 'instrumentalness': 6.35e-06, 'liveness': 0.137, 'valence': 0.486, 'tempo': 186.003}


In [145]:
import pickle

In [146]:
# Load scaler and XGBoost model
base_path = '../Models/'
with open(base_path+'scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open(base_path+'xgb.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

In [147]:
pd.DataFrame(audio_features, index=[0]).head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.679,0.587,7,-7.015,1,0.276,0.141,0.000006,0.137,0.486,186.003


In [148]:
# Function to preprocess input features using scaler
def preprocess_input(audio_features):
    # Convert audio features to numpy array
    # features_array = np.array(list(audio_features.values())).reshape(1, -1)
    scaled_features = scaler.transform(pd.DataFrame(audio_features, index=[0]))
    return scaled_features

In [149]:
# Function to make prediction using XGBoost model
def predict(features):
    return xgb_model.predict(features)[0]

In [150]:
# Preprocess input features
scaled_features = preprocess_input(audio_features)

In [151]:
# Make prediction
genre = predict(scaled_features)

In [152]:


with open(base_path+'encoder.pkl', 'rb') as f:
    genre_map = pickle.load(f)
print("Predicted Genre:", genre_map.inverse_transform([int(genre)])[0])

Predicted Genre: Rap


In [153]:
with open(base_path+'label_encoder.pkl', 'rb') as f:
    artist_name_map = pickle.load(f)
    
artists = df['artist_name'].unique()

print(artist_name_map.inverse_transform(artists))


['A Thousand Horses' 'Water Liars' 'Chambao' ... 'Steve Angello'
 'Sebastian Ingrosso' 'Man Man']


In [154]:
print(artist_name_map.transform([artist_name]))

[2718]


In [155]:
# Create DataFrame
data_df = pd.DataFrame({
    "artist_pop": [artist_pop],
    "popularity": [popularity],
    "year": [release_date],
    "genre": [genre],
    "artist_name": [artist_name_map.transform([artist_name])[0]]
})

In [156]:
# Load the model
with open(base_path+'birch.pkl', 'rb') as f:
    birch_model = pickle.load(f)

In [157]:
# Use the model to predict clusters
cluster = birch_model.predict(data_df)
print("Birch Clusters:", cluster)

Birch Clusters: [0]


In [158]:
#recommendation system
from sklearn.metrics.pairwise import cosine_similarity

#find nearest neighbors using cosine similarity
def find_nearest_neighbors(data, index, n_neighbors=5):
    similarities = cosine_similarity(data, data[index:index+1]).flatten()
    similar_indices = similarities.argsort()[-n_neighbors-1:-1]
    return similar_indices

# Function to recommend tracks
def recommend_tracks(data, cluster, n_recommendations=5):
    # Filter data for the same cluster
    cluster_data = data[data['cluster'] == cluster]
    cluster_data=cluster_data.drop('cluster', axis=1)
    features=['artist_name','artist_pop','popularity','year','genre']
    # Find nearest neighbors
    similar_indices = find_nearest_neighbors(cluster_data[features], 0, n_recommendations)
    # Return recommendations
    return cluster_data.iloc[similar_indices]

# Example usage
recommendations = recommend_tracks(df, cluster[0])
print("Recommendations for the track:")
print(recommendations[['track_name', 'artist_name', 'popularity', 'genre', 'artist_pop', 'year', ]])


Recommendations for the track:
            track_name  artist_name  popularity  genre  artist_pop  year
2807      Hit The Road         2912    0.239130      1        0.26  2014
281   Who I Am (Chyna)         2900    0.217391      1        0.62  2005
2200         Right Now         2913    0.000000      1        0.48  2014
3394        Me Acuerdo         2884    0.000000      1        0.25  1994
524          Fireworks         2913    0.000000      1        0.48  2013
